# Training TransE model in Ampligraph
07/08/20. HC contains ~30% people/orgs connected to Wikidata.

## 1. Import data

In [1]:
from ampligraph.datasets import load_from_csv
import numpy as np

In [ ]:
csv_folder = "/Volumes/Kalyan_SSD/SMG/"
csv_name = "hc_dump_070820.csv"

X = load_from_csv(csv_folder, csv_name, sep='\t')
X[:5, ]

In [ ]:
# too much for my RAM
# X = X.astype(np.str)

# np.unique below doesn't work with a mix of str & float
# entities = np.unique(np.concatenate([X[:,0], X[:,2]]))
# relations = np.unique(X[:,1])
# relations[0:5]

## 2. Train-test split

In [3]:
from ampligraph.evaluation import train_test_split_no_unseen

In [4]:
X_train, X_test = train_test_split_no_unseen(X, test_size=100) 

TypeError: '<' not supported between instances of 'float' and 'str'